In [6]:
!pip uninstall numpy -y
!pip install --upgrade numpy
!pip install --force-reinstall gensim scikit-learn
!pip install nlpaug

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 k

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 9.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv("drone_prompts_varied_2000.csv", delimiter=';')

df["category"].unique()

array(['строительство', 'сельское хозяйство', 'Горное дело', 'геодезия',
       'дорожное хозяйство'], dtype=object)

In [7]:
!pip install nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import random
from tqdm import tqdm

char_aug = nac.KeyboardAug(aug_char_p=0.2)
synonym_aug = naw.ContextualWordEmbsAug(
    model_path='DeepPavlov/rubert-base-cased',
    action="substitute",
    device='cpu')

def augment_text(text):
    if random.random() < 0.5:
        return char_aug.augment(text)
    else:
        return synonym_aug.augment(text)

tqdm.pandas()
df["sentence"] = df["sentence"].progress_apply(augment_text)

100%|██████████| 2001/2001 [24:59<00:00,  1.33it/s]


In [12]:
df["sentence"] = df["sentence"].apply(lambda x: x[0])

In [13]:
nltk.download("stopwords")
stop_words = set(stopwords.words("russian"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^а-яa-z\s]", "", text)
    tokens = text.split()
    tokens = [w for w in tokens if w not in stop_words and len(w) > 2]
    return " ".join(tokens)

df["sentence"] = df["sentence"].apply(clean_text)
class Vocab:
    def __init__(self, texts, min_freq=3):
        counter = Counter()
        for text in texts:
            counter.update(text.split())
        self.word2idx = {"<PAD>": 0, "<UNK>": 1}
        for word, freq in counter.items():
            if freq >= min_freq:
                self.word2idx[word] = len(self.word2idx)
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

    def encode(self, text, max_len=20):
        tokens = text.split()
        ids = [self.word2idx.get(word, 1) for word in tokens][:max_len]
        return ids + [0] * (max_len - len(ids))

    def vocab_size(self):
        return len(self.word2idx)

vocab = Vocab(df["sentence"])
le = LabelEncoder()
df["label"] = le.fit_transform(df["category"])
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx]), torch.tensor(self.labels[idx])

X = [vocab.encode(text) for text in df["sentence"]]
y = df["label"].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int,
        aggregation_type: str = 'max'
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.3)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1)
        else:
            raise ValueError("Invalid aggregation_type")

        output = self.dropout(self.linear(self.non_lin(output)))
        prediction = self.projection(self.non_lin(output))

        return prediction



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CharLM(
    hidden_dim=128,
    vocab_size=vocab.vocab_size(),
    num_classes=len(le.classes_),
    aggregation_type="max").to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(y_batch.numpy())

acc = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {acc}")
print(classification_report(all_labels, all_preds, target_names=le.classes_))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1 - Loss: 63.8381
Epoch 2 - Loss: 12.3747
Epoch 3 - Loss: 2.3867
Epoch 4 - Loss: 0.8673
Epoch 5 - Loss: 0.4600
📊 Accuracy: 0.9950
                    precision    recall  f1-score   support

       Горное дело       0.99      0.99      0.99        76
          геодезия       1.00      1.00      1.00        90
дорожное хозяйство       0.99      1.00      0.99        70
сельское хозяйство       1.00      0.99      0.99        74
     строительство       1.00      1.00      1.00        91

          accuracy                           1.00       401
         macro avg       0.99      0.99      0.99       401
      weighted avg       1.00      1.00      1.00       401



In [14]:
torch.save(model, "rnn_model.pt")
import joblib
joblib.dump(vocab, "vocab.pkl")
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']

In [30]:
torch.save(model.state_dict(), 'model_weights.pth')

In [15]:
model = torch.load("rnn_model.pt", weights_only=False, map_location="cpu")
model.eval()
vocab = joblib.load("vocab.pkl")
label_encoder = joblib.load("label_encoder.pkl")

In [26]:
text = "ужен дрон для оценки качества асфальта на трассах"
text = clean_text(text)
encoded = vocab.encode(text)
input_tensor = torch.tensor([encoded])
with torch.no_grad():
    output = model(input_tensor)
    pred = torch.argmax(output, dim=1).item()
    category = label_encoder.inverse_transform([pred])[0]

In [27]:
category

'дорожное хозяйство'

In [28]:
! pip freeze > requirements.txt

In [29]:
df.to_csv('aug_data.csv', index=False)